In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from scipy.io import loadmat
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split

from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt


In [2]:
# Importing the Data                                                                       Shape

Endg_Truth = loadmat('end6_groundTruth.mat')      
Ground_Truth = loadmat('groundTruth.mat')               
Urban = loadmat('Urban_R162.mat')                 
image = Urban['Y'].T                                                                #    94249*162  
groundTruth = Ground_Truth ['groundTruth']                                          #     307*307
image_shape = (Ground_Truth ['nRow'], Ground_Truth ['nCol'], image.shape[1])


def hyper_to_2D(img):
    """
    
    Converts a 3D HSI cube (m x n x p) to a 2D matrix of points (N X p)
+   N = m*n, p := number of bands

    """
    img = np.array(img)
    ndim = img.ndim

    if ndim not in [2, 3]:
      raise "invalid dimention. ndim = 2 or 3"

    if ndim == 2:
      m, n = np.shape(img)
      
      hyper_img = np.empty((m,n,1))
      hyper_img[:,:,0] = img

      return hyper_to_2D(hyper_img).ravel()

    m, n, p = np.shape(img)
    N = n * m
    
    dataset = img.T.reshape((p, N)).T

    if p == 1:
      return dataset.ravel()
    
    return dataset

image = image
groundTruth = hyper_to_2D(groundTruth)

print(image.shape)
print(groundTruth.shape)


(94249, 162)
(94249,)


In [3]:
# Standardization
# Split into train (2/3) and test (1/3) sets

image = StandardScaler().fit_transform(image)
seed = 40
X_train, X_test, y_train, y_test = train_test_split(image, groundTruth, test_size = 0.30 , train_size = 0.70 , random_state = seed , stratify=groundTruth)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)


(65974, 162)
(28275, 162)
(65974,)
(28275,)


In [4]:
# sequential forward Feature Selection

SFS_FS = [14, 47, 61, 76, 106, 107, 115, 129, 142, 145]


In [5]:
# Tune parameters
# kernel sigmoid

G=[0.1,1,10]
Coef=[0.25,0.5,4]
accuracy_test = []
accuracy_train = []

for coef in Coef:
    for g in G:
    
        svm = SVC(kernel= 'sigmoid', random_state=1, C=100, gamma=g,coef0=coef)
        svm.fit(X_train[:,  SFS_FS],y_train)
        predict_Ltest = svm.predict(X_test[:,  SFS_FS])
        predict_Ltrain = svm.predict(X_train[:,  SFS_FS])
        a1=accuracy_score(y_test,predict_Ltest)
        a2=accuracy_score(y_train,predict_Ltrain)
        accuracy_test.append([g,coef,a1])
        print('g=',g,'coef=',coef,'accuracy test=',a1)
        accuracy_train.append([g,coef,a2])
        print('g=',g,'coef=',coef,'accuracy train=',a2)

g= 0.1 coef= 0.25 accuracy test= 0.5426348364279399
g= 0.1 coef= 0.25 accuracy train= 0.5476248218995362
g= 1 coef= 0.25 accuracy test= 0.38854111405835545
g= 1 coef= 0.25 accuracy train= 0.3906084214993785
g= 10 coef= 0.25 accuracy test= 0.45389920424403185
g= 10 coef= 0.25 accuracy train= 0.4557552975414557
g= 0.1 coef= 0.5 accuracy test= 0.4784438549955791
g= 0.1 coef= 0.5 accuracy train= 0.48294782793221575
g= 1 coef= 0.5 accuracy test= 0.397736516357206
g= 1 coef= 0.5 accuracy train= 0.40173401643071516
g= 10 coef= 0.5 accuracy test= 0.4506808134394341
g= 10 coef= 0.5 accuracy train= 0.45254191044957104
g= 0.1 coef= 4 accuracy test= 0.46436781609195404
g= 0.1 coef= 4 accuracy train= 0.4641222299693819
g= 1 coef= 4 accuracy test= 0.2549955791335102
g= 1 coef= 4 accuracy train= 0.25073513808470005
g= 10 coef= 4 accuracy test= 0.46783377541998233
g= 10 coef= 4 accuracy train= 0.47141298087125233
